In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import sys
import iprompt.data
from transformers import AutoTokenizer, OPTForCausalLM
from iprompt.data import TASKS_GALACTICA
from imodelsx import explain_dataset_iprompt, get_add_two_numbers_dataset

# BBBP

In [3]:
task = TASKS_GALACTICA['bbbp']
df = task['gen_func']()
input_strings = df['input'].values
output_strings = df['output'].values

In [6]:
# get a simple dataset of adding two numbers

# explain the relationship between the inputs and outputs
# with a natural-language prompt string
prompts, metadata = explain_dataset_iprompt(
    input_strings=input_strings,
    output_strings=output_strings,
    # checkpoint='EleutherAI/gpt-j-6B', # which language model to use
    checkpoint="facebook/galactica-6.7b", # which language model to use
    preprefix='To get the answer from the compound formala,',
    num_learned_tokens=10, # how long of a prompt to learn
    n_shots=5, # shots per example
    n_epochs=15, # how many epochs to search
    verbose=0, # how much to print
    llm_float16=False, # whether to load the model in float_16
)

Using eos_token, but it is not set yet.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1244372/460054166.py:5 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1244372/460054166.py'                       │
│                                                                                                  │
│ /home/chansingh/imodelsX/imodelsx/iprompt/api.py:431 in explain_dataset_iprompt                  │
│                                                                                                  │
│   428 │   loss_func = PrefixLoss(gamma=gamma, tokenizer=tokenizer)                               │
│   429 │                                                                                          │
│   430 │   if model_cls == 'iprompt':                                                             │
│ ❱ 431 │   │   model = iPrompt(                                                                   │
│   432 │   │   │   loss_func=loss_func,                                                           │
│   433 │   │   │   model=lm,                                                                      │
│   434 │   │   │   tokenizer=tokenizer,                                                           │
│                                                                                                  │
│ /home/chansingh/imodelsX/imodelsx/iprompt/ipromptx.py:47 in __init__                             │
│                                                                                                  │
│    44 │   │   args.autoprompt_init_strategy = None                                               │
│    45 │   │   args.save_dir_unique = '.'                                                         │
│    46 │   │   args.max_length = max_length                                                       │
│ ❱  47 │   │   super().__init__(                                                                  │
│    48 │   │   │   args=args, loss_func=loss_func, model=model, tokenizer=tokenizer, preprefix=   │
│    49 │   │   )                                                                                  │
│    50 │   │   self.tokenizer = tokenizer                                                         │
│                                                                                                  │
│ /home/chansingh/imodelsX/imodelsx/iprompt/autoprompt.py:34 in __init__                           │
│                                                                                                  │
│    31 │   │   │   tokenizer: transformers.PreTrainedTokenizer,                                   │
│    32 │   │   │   preprefix: str = ''                                                            │
│    33 │   │   ):                                                                                 │
│ ❱  34 │   │   super().__init__(                                                                  │
│    35 │   │   │   args=args, loss_func=loss_func, model=model, tokenizer=tokenizer, preprefix=   │
│    36 │   │   )                                                                                  │
│    37 │   │   # AutoPrompt-specific parameters.                                                  │
│                                                                                                  │
│ /home/chansingh/imodelsX/imodelsx/iprompt/hotflip.py:52 in __init__                              │
│                                                                                                  │
│    49 │   │   preprefix_ids = [self.tokenizer.bos_token_id]                                      │
│    50 │   │   if preprefix:                                                                      │
│    51 │   │   │   preprefix_ids.extend(self.tokenizer.encod

# Example

In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-6.7b")
model = OPTForCausalLM.from_pretrained("facebook/galactica-6.7b", device_map="auto", torch_dtype=torch.float16)

In [3]:
def gen(model, input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=200)
    return tokenizer.decode(outputs[0])

In [4]:
gen(model, "[START_I_SMILES]")

'[START_I_SMILES]CC1=CC=C(C=C1)C2=NN=C(N2C3=CC=CC=C3)SCC(=O)NC4=CC=CC=C4OC[END_I_SMILES]\n\n### Molecular Formula\n\nC24H22N4O2S\n\n## Chemical and Physical Properties\n\nThe following are chemical properties for 2-[[4,5-bis(p-tolyl)-1,2,4-triazol-3-yl]sulfanyl]-N-(2-methoxyphenyl)acetamide.\n\n### Computed Properties\n\n| Property Name | Property Value\n| --- | ----------- |\n| Molecular Weight | 430.5\n| XLogP3-AA Log P | 4.9\n| Hydrogen Bond Donor Count | 1\n| Hydrogen Bond Acceptor Count |'